## SVM Classification

In [1]:
from numpy import linalg
from menpo import io as mio
from menpo.visualize import print_dynamic
from sklearn.utils.fixes import bincount
import itertools
import warnings
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from dAAMs.lda import lda, predict, chunk, n_fold_generate
from sklearn import svm

%matplotlib inline

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.
Using Theano backend.


In [2]:
def cross_validation(fdata):
    accuracy = 0
    for j,test_set in enumerate(fdata):
        print('processing {}/{} fold'.format(j+1, len(fdata)))
        
        train_set = reduce(lambda x,y: x+y, fdata[:j] + fdata[j+1:])
        
        train_X = map(lambda x:x[-1], train_set)
        train_y = map(lambda x:x[0], train_set)
        test_X = map(lambda x:x[-1], test_set)
        test_y = map(lambda x:x[0], test_set)
        
        clf = svm.SVC(C=1)
        clf.fit(train_X,train_y)
        
        pred_y = clf.predict(test_X)
        
        acc = np.sum(test_y == pred_y) / float(len(pred_y))
        
        print '{}/{} fold acc: {}'.format(j+1, len(fdata), acc)
        
        accuracy += acc
        
        del train_set, train_X, train_y, test_X, test_y, pred_y, clf
        
        
    return accuracy / float(len(fdata))

### cross validation test

#### WPUTEDB

In [3]:
data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/LDA-WPUTEDB-Data-dsift-patch.pkl')

In [4]:
fdata = n_fold_generate(data, n_fold=4)

In [ ]:
cross_validation(fdata)

processing 1/4 fold


#### VGG Ear

In [3]:
data = mio.import_pickle('/homes/yz4009/wd/PickleModel/EarRecognition/.pkl')

In [4]:
fdata = n_fold_generate(data, n_fold=4)

In [ ]:
cross_validation(fdata)

### test 

In [5]:
X = []
y = []
for idx,img in reduce(lambda x,y:x+y, fdata[:3]):
    X.append(img)
    y.append(idx)

In [6]:
tx = []
ty = []
for idx,img in fdata[-1]:
    tx.append(img)
    ty.append(idx)

In [15]:
%%time
clf = svm.SVC(C=125)
clf.fit(X,y)

CPU times: user 2h 55min 9s, sys: 50.2 s, total: 2h 55min 59s
Wall time: 3h 57min 1s


In [16]:
py = clf.predict(tx)

In [17]:
(py == ty).sum() / float(len(fdata[-1]))

0.70663562281722936

In [18]:
"c1,c2: 0.7020","c5:0.7066 ", "c25: 0.7066", "c125: 0.7066"

('c1,c2: 0.7020', 'c5:0.7066 ', 'c25: 0.7066', 'c125: 0.7066')